# An Introduction to LlamaIndex Query Pipelines

## Overview
LlamaIndex provides a declarative query API that allows you to chain together different modules in order to orchestrate simple-to-advanced workflows over your data.

This is centered around our `QueryPipeline` abstraction. Load in a variety of modules (from LLMs to prompts to retrievers to other pipelines), connect them all together into a sequential chain or DAG, and run it end2end.

**NOTE**: You can orchestrate all these workflows without the declarative pipeline abstraction (by using the modules imperatively and writing your own functions). So what are the advantages of `QueryPipeline`? 

- Express common workflows with fewer lines of code/boilerplate
- Greater readability
- Greater parity / better integration points with common low-code / no-code solutions (e.g. LangFlow)
- [In the future] A declarative interface allows easy serializability of pipeline components, providing portability of pipelines/easier deployment to different systems.

## Cookbook

In this cookbook we give you an introduction to our `QueryPipeline` interface and show you some basic workflows you can tackle.

- Chain together prompt and LLM
- Chain together query rewriting (prompt + LLM) with retrieval
- Chain together a full RAG query pipeline (query rewriting, retrieval, reranking, response synthesis)

## Setup

Here we setup some data + indexes (from PG's essay) that we'll be using in the rest of the cookbook.

In [1]:
from llama_index.query_pipeline.query import QueryPipeline

In [2]:
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate
from llama_index import (
    VectorStoreIndex, 
    ServiceContext,
    SimpleDirectoryReader,
    load_index_from_storage
)

In [3]:
reader = SimpleDirectoryReader("../data/paul_graham")

In [4]:
docs = reader.load_data()

In [5]:
import os
from llama_index.storage import StorageContext

if not os.path.exists("storage"):
    index = VectorStoreIndex.from_documents(docs)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

## 1. Chain Together Prompt and LLM 

In this section we show a super simple workflow of chaining together a prompt with LLM.

In [6]:
# try chaining basic prompts
prompt_str = "Please generate related movies to {movie_name}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model='gpt-3.5-turbo')

p = QueryPipeline(chain=[prompt_tmpl, llm])

In [7]:
output = p.run(movie_name="The Departed")

Running module bb10c7ed-4597-460b-a00a-99311fbc47b2 with input {'movie_name': 'The Departed'}
Running module dbe7261f-75b4-4e68-9c3c-775d1d737112 with input {'messages': 'Please generate related movies to The Departed'}


In [8]:
print(str(output))

assistant: 1. Infernal Affairs (2002) - The Departed is actually a remake of this Hong Kong crime thriller. It follows a similar storyline of undercover cops infiltrating the criminal underworld.

2. The Town (2010) - Directed by Ben Affleck, this crime drama revolves around a group of bank robbers in Boston. It explores themes of loyalty, betrayal, and the blurred lines between law enforcement and criminals.

3. American Gangster (2007) - Based on a true story, this crime film follows the rise and fall of a Harlem drug lord and the detective determined to bring him down. It delves into the corrupt world of organized crime and the efforts to dismantle it.

4. Training Day (2001) - Denzel Washington won an Academy Award for his portrayal of a corrupt narcotics detective in this intense crime thriller. The movie explores the moral dilemmas faced by law enforcement officers and the consequences of their actions.

5. The Departed (2006) - Although it's the same movie, it's worth mentioning

### Try chaining llm, prompt, retriever

In [9]:
from llama_index.postprocessor import CohereRerank
prompt_str = "Please generate a question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl = PromptTemplate(prompt_str)

llm = OpenAI(model='gpt-3.5-turbo')
retriever = index.as_retriever(similarity_top_k=5)
p = QueryPipeline(chain=[prompt_tmpl, llm, retriever])

In [10]:
nodes = p.run(topic="college")
len(nodes)

Running module 8232b621-4381-4bd7-95f3-0235ee8f8411 with input {'topic': 'college'}
Running module 971576fe-fdca-427c-a59f-42e03bfee1c4 with input {'messages': "Please generate a question about Paul Graham's life regarding the following topic college"}
Running module d4c48953-077a-4467-99db-b31fa344177a with input {'input': ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="What impact did Paul Graham's college experience have on his career and entrepreneurial journey?", additional_kwargs={}), raw={'id': 'chatcmpl-8e87jQN9JiWAXQRuOJvLUXnpivNY5', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="What impact did Paul Graham's college experience have on his career and entrepreneurial journey?", role='assistant', function_call=None, tool_calls=None))], 'created': 1704575039, 'model': 'gpt-3.5-turbo-0613', 'object': 'chat.completion', 'system_fingerprint': None, 'usage': CompletionUsage(completion_tokens

5

### Try chaining llm, prompt, retriever, reranker, response synthesizer

Here we can't use `chain` syntax because we have to draw up a DAG.

In [11]:
from llama_index.postprocessor import CohereRerank
from llama_index.response_synthesizers import TreeSummarize
from llama_index import ServiceContext

# define modules
prompt_str = "Please generate a question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model='gpt-3.5-turbo')
retriever = index.as_retriever(similarity_top_k=3)
reranker = CohereRerank()
summarizer = TreeSummarize(service_context=ServiceContext.from_defaults(llm=llm))

In [12]:
# define query pipeline
p = QueryPipeline()
p.add_modules({
	"llm": llm,
	"prompt_tmpl": prompt_tmpl,
	"retriever": retriever,
    "summarizer": summarizer,
    "reranker": reranker
})
p.add_link("prompt_tmpl", "llm")
p.add_link("llm", "retriever")
p.add_link("retriever", "reranker", dest_key="nodes")
p.add_link("llm", "reranker", dest_key="query_str")
p.add_link("reranker", "summarizer", dest_key="nodes")
p.add_link("llm", "summarizer", dest_key="query_str")

In [13]:
response = p.run(topic="YC")

Running module prompt_tmpl with input {'topic': 'YC'}
Running module llm with input {'messages': "Please generate a question about Paul Graham's life regarding the following topic YC"}
Running module retriever with input {'input': ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='What role did Paul Graham play in the founding and development of Y Combinator (YC)?', additional_kwargs={}), raw={'id': 'chatcmpl-8e87nxAi4dy9eB80Cr7MDbAsQk84B', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='What role did Paul Graham play in the founding and development of Y Combinator (YC)?', role='assistant', function_call=None, tool_calls=None))], 'created': 1704575043, 'model': 'gpt-3.5-turbo-0613', 'object': 'chat.completion', 'system_fingerprint': None, 'usage': CompletionUsage(completion_tokens=19, prompt_tokens=22, total_tokens=41)}, delta=None, additional_kwargs={})}
Running module reranker with input {'query

In [14]:
print(str(response))

Paul Graham played a significant role in the founding and development of Y Combinator (YC). He was one of the co-founders of YC and was actively involved in its early stages. He helped establish the Summer Founders Program (SFP) and was responsible for selecting and funding the initial batch of startups. Graham also played a key role in shaping the funding model for YC, based on previous deals and experiences. As YC grew, Graham's attention and focus shifted more towards YC, and he gradually reduced his involvement in other projects. However, he continued to work on YC's internal software and wrote essays about startups. Eventually, Graham decided to hand over the reins of YC to someone else and recruited Sam Altman as the new president of YC.
